In [1]:
import numpy as np
import torch
import torch.nn.functional as F

### Approximating within-softmax QK decomposition to bring dimensionality reduction outside softmax

In [ ]:
# can we approximate e^(AB) with e^A

### Can we approximate normalization coefficient of eigen-decomp attention matrix with that of original attention matrix?
If attention eigenvalue mass is preserved, and the eigen-attention matrix is faithful approximation, total attention mass per token should be the same

### Second Attempt

In [111]:
matrixSize = 4
A = np.random.rand(matrixSize, matrixSize)
B = np.dot(A, A.transpose())

In [112]:
B

array([[0.9980213 , 1.33010605, 0.45539975, 0.47717353],
       [1.33010605, 2.06308748, 0.78415537, 0.73757189],
       [0.45539975, 0.78415537, 0.40060215, 0.28996259],
       [0.47717353, 0.73757189, 0.28996259, 0.26989832]])

In [136]:
psd_ratio = 1  # test how error increases as Q becomes less PSD
d = 2
e = 4
l = 4
assert l >= d
arange_d = torch.arange(d, 0, -1)
psd_matrix = torch.Tensor(np.random.rand(d, d))
psd_matrix = torch.matmul(psd_matrix, psd_matrix)
psd_matrix = torch.cat([psd_matrix, torch.zeros(l - d, d)], dim=0)
q = torch.Tensor(np.random.rand(l, d)) * (1 - psd_ratio) + psd_matrix * psd_ratio
q_inv = torch.linalg.pinv(q)
k = torch.Tensor(np.random.rand(l, d)) * (1 - psd_ratio) + psd_matrix * psd_ratio

q, k = q.type(torch.float), k.type(torch.float)

print(q)
print(torch.linalg.matrix_rank(q))
print(q_inv @ q)
#print(q @ q_inv) # not valid

v = torch.Tensor(np.random.rand(l, d) / temp)

tensor([[0.7265, 0.0551],
        [0.8841, 1.0138],
        [0.0000, 0.0000],
        [0.0000, 0.0000]])
tensor(2)
tensor([[ 1.0000e+00, -5.9605e-08],
        [ 1.1921e-07,  1.0000e+00]])


In [119]:
A = torch.exp(torch.matmul(q, k.T))  # [ queries X keys ]
A.shape

torch.Size([4, 4])

In [158]:
a = np.random.rand(3,3)
b = np.random.rand(3, 1)
c = a / b
print(c)
np.linalg.pinv(np.diag(b[:,0])) @ a

[[0.42363572 0.24654444 0.43344927]
 [2.71414006 5.14193764 4.23819015]
 [0.07293728 0.37789997 0.21410929]]


array([[0.42363572, 0.24654444, 0.43344927],
       [2.71414006, 5.14193764, 4.23819015],
       [0.07293728, 0.37789997, 0.21410929]])

In [139]:
# compute decomposition
eigenvals, E = torch.linalg.eig(A)  # eigenvectors are columns of E
eigenvals, E = eigenvals[:e], E[:,:e]  # take only e-highest eigenvectors
Lambda = torch.diag(eigenvals)
E = E.type(torch.float32)

print(E)
# test
print(torch.log(E))
(q @ (q_inv @ torch.log(E)))
# E_hat = torch.matmul(q, torch.matmul(torch.linalg.pinv(q), torch.log(E)))
# E_hat2 = q @ torch.linalg.lstsq(q, torch.log(E)).solution
# print(E)
# print(torch.exp(E_hat))
# print(torch.exp(E_hat2))

tensor([[ 6.2900e-01,  7.6752e-01, -1.2362e-01,  5.2490e-08],
        [ 4.6785e-01, -2.4672e-01,  8.4868e-01, -3.3643e-07],
        [ 4.3902e-01, -4.1836e-01, -3.6364e-01, -7.0711e-01],
        [ 4.3902e-01, -4.1836e-01, -3.6364e-01,  7.0711e-01]])
tensor([[ -0.4636,  -0.2646,      nan, -16.7626],
        [ -0.7596,      nan,  -0.1641,      nan],
        [ -0.8232,      nan,      nan,      nan],
        [ -0.8232,      nan,      nan,  -0.3466]])


tensor([[-0.4636,     nan,     nan,     nan],
        [-0.7596,     nan,     nan,     nan],
        [ 0.0000,     nan,     nan,     nan],
        [ 0.0000,     nan,     nan,     nan]])

In [140]:
np.log(-7.0711e-01)

/tmp/ipykernel_2419/674491707.py:1: RuntimeWarning: invalid value encountered in log
  np.log(-7.0711e-01)


nan

In [280]:
E @ Lambda @ torch.linalg.pinv(E)

tensor([[1.0035+0.j, 1.0106+0.j, 1.0177+0.j, 1.0249+0.j, 1.0321+0.j],
        [1.0106+0.j, 1.0466+0.j, 1.0840+0.j, 1.1228+0.j, 1.1628+0.j],
        [1.0177+0.j, 1.0840+0.j, 1.1547+0.j, 1.2300+0.j, 1.3102+0.j],
        [1.0249+0.j, 1.1228+0.j, 1.2300+0.j, 1.3475+0.j, 1.4762+0.j],
        [1.0321+0.j, 1.1628+0.j, 1.3102+0.j, 1.4762+0.j, 1.6632+0.j]])

In [278]:
A

tensor([[1.0035, 1.0106, 1.0177, 1.0249, 1.0321],
        [1.0106, 1.0467, 1.0840, 1.1228, 1.1629],
        [1.0177, 1.0840, 1.1547, 1.2300, 1.3102],
        [1.0249, 1.1228, 1.2300, 1.3475, 1.4762],
        [1.0321, 1.1629, 1.3102, 1.4762, 1.6632]])

In [240]:
A

tensor([[ 3.8236, 11.6055],
        [ 3.0818,  5.9459]])

### First Attempt

In [212]:
temp = 1
d = 2
e = 5
l = 5
q = torch.Tensor(np.random.rand(l, d) / temp)
k = torch.Tensor(np.random.rand(l, d) / temp)
v = torch.Tensor(np.random.rand(l, d) / temp)
A_pre = torch.matmul(q, k.T)
A_norm = torch.exp(A_pre).sum(dim=1)
#A = F.softmax(A_pre, dim=1)  # Experiment with non-normalized case
A = torch.exp(A_pre)
print(f"A:\n{A}")
out = torch.matmul(A, v)
print(f"Output:\n{out}")

A:
tensor([[1.7639, 1.7609, 1.1244, 1.7304, 1.3712],
        [2.0524, 2.3596, 1.1748, 2.2233, 2.0929],
        [2.3386, 2.5569, 1.2016, 2.4313, 1.9976],
        [2.4823, 2.6884, 1.2155, 2.5579, 2.0196],
        [1.7632, 2.0020, 1.1372, 1.9017, 1.8649]])
Output:
tensor([[3.3751, 6.1221],
        [4.4014, 7.8028],
        [4.6564, 8.2506],
        [4.8488, 8.5767],
        [3.8390, 6.8625]])


In [210]:
# compute decomposition
eigenvals, E = torch.linalg.eig(A)  # eigenvectors are columns of E
eigenvals, E = eigenvals[:e], E[:,:e]  # take only e-highest eigenvectors
Lambda = torch.diag(eigenvals)

# compute new k from q, eigenvectors, and normalization coefficients of original attention matrix
#k_hat = torch.matmul(torch.linalg.pinv(q.type(torch.complex64)), torch.log(E * A_norm[:,None])).T.type(torch.float32)
k_hat = torch.matmul(torch.linalg.pinv(q.type(torch.complex64)), torch.log(E)).T.type(torch.float32)

# compute new v from v, eigenvectors, and lambda
v_hat = torch.matmul(torch.matmul(Lambda, torch.linalg.pinv(E)), v.type(torch.complex64)).type(torch.float32)

In [211]:
# compute new output
print(k_hat)
A_pre = torch.matmul(q, k_hat.T)
print(f"New A_pre:\n{A_pre}")
A_hat = F.softmax(torch.matmul(q, k_hat.T), dim=1)
print(f"New A:\n{A_hat}")
out_hat = torch.matmul(A_hat, v_hat)
print(f"New Output:\n{out_hat}")

tensor([[-0.5776, -0.6632],
        [-1.2459, -0.2738],
        [-1.2459, -0.2738],
        [-0.6523, -1.3320],
        [-2.4310,  0.3251]])
New A_pre:
tensor([[-0.2907, -0.1502, -0.1502, -0.5687,  0.0613],
        [-0.9064, -0.8535, -0.8535, -1.5790, -0.8469],
        [-0.6065, -1.0570, -1.0570, -0.8116, -1.8760],
        [-0.9424, -1.2057, -1.2057, -1.4811, -1.7382],
        [-0.2090, -0.2059, -0.2059, -0.3595, -0.2197]])
New A:
tensor([[0.1824, 0.2100, 0.2100, 0.1382, 0.2594],
        [0.2137, 0.2253, 0.2253, 0.1091, 0.2268],
        [0.2967, 0.1891, 0.1891, 0.2417, 0.0834],
        [0.2800, 0.2152, 0.2152, 0.1634, 0.1263],
        [0.2059, 0.2066, 0.2066, 0.1772, 0.2037]])
New Output:
tensor([[-1.3402, -2.5358],
        [-1.5631, -2.9675],
        [-2.1318, -4.1095],
        [-2.0231, -3.8810],
        [-1.5032, -2.8593]])


In [207]:
# compute new output
print(k_hat)
A_pre = torch.matmul(q, k_hat.T)
print(f"New A_pre:\n{A_pre}")
A_hat = F.softmax(torch.matmul(q, k_hat.T), dim=1)
print(f"New A:\n{A_hat}")
out_hat = torch.matmul(A_hat, v_hat)
print(f"New Output:\n{out_hat}")

tensor([[ 0.8786, -2.0016],
        [ 0.7563, -2.1765],
        [ 3.7824, -5.3803],
        [ 0.4248, -2.1134],
        [ 1.7735, -3.4110]])
New A_pre:
tensor([[-0.6289, -0.7102, -1.5018, -0.7308, -1.0350],
        [-0.7243, -0.9397, -0.8613, -1.1491, -1.0233],
        [-0.8518, -0.9481, -2.1340, -0.9545, -1.4213],
        [-0.9654, -1.1892, -1.5990, -1.3718, -1.4515],
        [-0.7705, -1.0297, -0.7016, -1.2984, -1.0468]])
New A:
tensor([[0.2558, 0.2358, 0.1069, 0.2310, 0.1704],
        [0.2455, 0.1979, 0.2141, 0.1605, 0.1820],
        [0.2737, 0.2486, 0.0759, 0.2470, 0.1548],
        [0.2769, 0.2214, 0.1470, 0.1844, 0.1703],
        [0.2386, 0.1841, 0.2556, 0.1407, 0.1810]])
New Output:
tensor([[-0.2635, -0.3403],
        [-0.2533, -0.3273],
        [-0.2822, -0.3640],
        [-0.2861, -0.3690],
        [-0.2463, -0.3183]])


In [197]:
k_hat

tensor([[-0.5063,  2.4431],
        [-0.6286,  2.2682],
        [ 2.3975, -0.9356],
        [-0.9601,  2.3313],
        [ 0.3886,  1.0337]])

In [199]:
k_hat

tensor([[ 0.8786, -2.0016],
        [ 0.7563, -2.1765],
        [ 3.7824, -5.3803],
        [ 0.4248, -2.1134],
        [ 1.7735, -3.4110]])

In [192]:
print(torch.log(E).type(torch.float))
print(torch.log(E * A_norm[:,None]).type(torch.float))

tensor([[-0.8047, -0.6859, -1.1466, -1.7738, -2.0176],
        [-0.8047, -1.2718, -1.1452, -0.6483, -0.7625],
        [-0.8047, -0.6890, -2.0405, -0.2543, -0.3628],
        [-0.8047, -1.6163, -2.1543, -1.4873, -2.2536],
        [-0.8047, -0.4888, -0.1323, -1.5474, -0.6553]])
tensor([[ 1.0170,  1.1358,  0.6751,  0.0479, -0.1959],
        [ 1.3939,  0.9268,  1.0535,  1.5503,  1.4361],
        [ 1.0563,  1.1720, -0.1795,  1.6067,  1.4981],
        [ 1.4145,  0.6029,  0.0649,  0.7319, -0.0344],
        [ 1.5231,  1.8391,  2.1955,  0.7804,  1.6725]])


In [184]:
torch.matmul(torch.matmul(E, Lambda), torch.linalg.pinv(E))  # verify this is identical to A

tensor([[0.2128+0.j, 0.1722+0.j, 0.1904+0.j, 0.2263+0.j, 0.1984+0.j],
        [0.2745+0.j, 0.1418+0.j, 0.1555+0.j, 0.2239+0.j, 0.2043+0.j],
        [0.2107+0.j, 0.1659+0.j, 0.1906+0.j, 0.2363+0.j, 0.1964+0.j],
        [0.2676+0.j, 0.1378+0.j, 0.1577+0.j, 0.2352+0.j, 0.2018+0.j],
        [0.2957+0.j, 0.1321+0.j, 0.1449+0.j, 0.2225+0.j, 0.2048+0.j]])

In [137]:
k_hat

tensor([[3.3408+0.0000e+00j, 2.5500+8.7651e-04j, 2.2108-5.7629e-01j,
         2.2108+5.7629e-01j, 2.0674+9.8414e-02j],
        [4.7148+0.0000e+00j, 4.8400+9.6306e-01j, 4.5572+5.0659e-01j,
         4.5572-5.0659e-01j, 4.6560+3.7260e-01j]])

In [141]:
 torch.matmul(torch.linalg.pinv(q.type(torch.complex64)), torch.log(E * A_norm[:,None]))

tensor([[3.3408+0.0000e+00j, 2.5500+8.7655e-04j, 2.2108-5.7629e-01j,
         2.2108+5.7629e-01j, 2.0674+9.8414e-02j],
        [4.7148+0.0000e+00j, 4.8400+9.6306e-01j, 4.5572+5.0659e-01j,
         4.5572-5.0659e-01j, 4.6560+3.7260e-01j]])

In [130]:
torch.log(E * A_norm[:,None])

tensor([[ 9.9680+3.1416j,  9.6554+3.1416j,  7.7604+3.1416j],
        [10.5996+3.1416j,  9.7819+0.0000j,  7.3967+3.1416j],
        [14.4327+3.1416j, 14.6182+0.0000j, 14.4411+0.0000j],
        [24.5688+3.1416j, 25.0835+0.0000j, 25.2253+0.0000j],
        [16.8817+3.1416j, 14.9674+3.1416j, 16.1924+0.0000j]])

In [126]:
k_hat.solution

tensor([[1.3275+0.0000j, 0.0615+0.5169j, 0.0615-0.5169j],
        [2.0710+0.0000j, 3.3681-0.8383j, 3.3681+0.8383j]])

In [108]:
np.linalg.matrix_rank(q)

3

In [110]:
E

tensor([[ 0.4472,  0.4465],
        [ 0.4472,  0.4221],
        [ 0.4472, -0.3806],
        [ 0.4472,  0.5325],
        [ 0.4472,  0.4406]])

In [91]:
torch.diag(E)

tensor([[ 1.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j],
        [ 0.0000+0.0000j, -0.2225+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j, -0.0216+0.0310j,  0.0000+0.0000j,
          0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j, -0.0216-0.0310j,
          0.0000+0.0000j],
        [ 0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,  0.0000+0.0000j,
          0.0255+0.0000j]])

In [ ]:
k_hat = torch.linalg.lstsq(q, torch.log(

In [ ]:
# Testing eigenvector dimensionality
E, V = torch.linalg.eig(A)  # eigenvectors are columns of V

In [81]:
torch.matmul(A.type(torch.complex64), V[:,:1])

tensor([[-0.4472+0.j],
        [-0.4472+0.j],
        [-0.4472+0.j],
        [-0.4472+0.j],
        [-0.4472+0.j]])

In [83]:
V[:,:1]

tensor([[-0.4472+0.j],
        [-0.4472+0.j],
        [-0.4472+0.j],
        [-0.4472+0.j],
        [-0.4472+0.j]])

Can we solve for k_opt from q and u_post? Initial guess: log(u_post) cross left inverse of q

In [46]:
k_opt

tensor([[1.8213, 2.0570, 2.2274],
        [1.0515, 2.4759, 1.7531]])

In [43]:
# does left inverse exist? check rank of q
np.linalg.matrix_rank(q) == d

True

In [44]:
# in general, q should have a left inverse
k_est, _, _, _ = np.linalg.lstsq(q, np.log(u_post), rcond=None)
k_est = k_est.T
k_est

array([[ 0.09949453, -0.31391796, -0.13661294],
       [-0.6703938 ,  0.10499796, -0.61092776]], dtype=float32)

In [45]:
# fail - let's construct normalization term from k_est and see if we can get closer

tensor([[1.8213, 2.0570, 2.2274],
        [1.0515, 2.4759, 1.7531]])